In [1]:
import pandas as pd
from glob import glob
from PIL import Image, ImageDraw
import cv2
import numpy
import matplotlib.pyplot as plt
import os
import random 
from ultralytics import YOLO
from tqdm import tqdm
%matplotlib inline


In [ ]:
def get_images_labels(path, folder_name):
    labels = glob(f'{path}/train_labels/train_labels/{folder_name}*')
    labels = sorted(labels)
    imgs = glob(f'{path}/data/train/{folder_name.split("_")[0]}/{folder_name}*')
    imgs = sorted(imgs)
    return labels, imgs

def generate_random_color():
    """Generate a bright, random color."""
    return tuple(random.randint(100, 255) for _ in range(3))  # RGB values


def stack_images_vertically(image_paths):
    '''Stack images vertically into a single PIL image cropping to the minimum width'''
    images = [Image.open(path) for path in image_paths]
    widths, heights = zip(*(i.size for i in images))
    min_width = min(widths)
    total_height = sum(heights)
    stacked_image = Image.new('RGB', (min_width, total_height))
    y_offset = 0
    for im in images:
        if im.width > min_width:
            im = im.crop((0,0, min_width, im.height))
        stacked_image.paste(im, (0, y_offset))
        y_offset += im.height
    return stacked_image


import os 
from PIL import Image


In [ ]:
import os
from PIL import Image
def prepare_stacked_data(image_paths, label_paths):
    '''
    Prepare a stacked image and combined mask string from matching image-label pairs

    Args:
    image_paths (list): List of paths to image files
    label_paths (list): List of paths to label files

    Return:
        tuple: (stacked_image, combined_masks_str)
            - stacked images: PIL image object of the stacked images.
            -combined_mask_str: string combining all mask annotations.
    '''

    # create a dictionary mapping the label base names to their full paths 
    label_map = {os.path.splitext(os.path.basename(lf))[0]: lf for lf in label_paths}
    matching_images = []
    matching_labels = []

    # match images with corresponding labels
    for image_path in image_paths:
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        if image_name in label_map:
            matching_images.append(image_path)
            matching_labels.append(label_map[image_name])

    #print feedback for debugging
    print(f'found {len(image_paths)} images and {len(label_paths)} labels')
    print